<a href="https://colab.research.google.com/github/Somya000/Data-Science/blob/main/Autism_Intervention_using_LLAMA_2_and_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing Libraries**

In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 71.8 MB/s eta 0:00:00


In [2]:
!pip install llama-index
%pip install llama-index-llms-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 8.0 MB/s eta 0:00:00


In [6]:
## Embedding:
!pip install install sentence_transformers

In [7]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [8]:
!pip install PyMuPDF nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 29.6 MB/s eta 0:00:00


**Data Preprocessing and Cleaning**

In [9]:
import os
import fitz
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import string

**Extract Text from PDFs**

In [10]:
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file using PyMuPDF (fitz)."""
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Failed to read {pdf_path}: {e}")
        return None


In [11]:
#Function to remove References
def remove_references(text):
    """Remove references from the text by identifying the 'References' section."""
    # Pattern to identify the start of the references section
    references_patterns = [
        r'\bReferences\b',
        r'\bBibliography\b',
        r'\bWorks Cited\b',
        r'\bCitations\b',
    ]
    for pattern in references_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            text = text[:match.start()]
            break
    return text


# Function to remove figures,tables and in-text citations
def clean_text(text):
    """
    Remove figure and table captions and related text from the text.
    """
    # Regular expression to find figure captions
    text = re.sub(r'Figure \d+:.*?(?=\n\n|\Z)', '', text, flags=re.DOTALL)
    # Regular expression to find table captions
    text = re.sub(r'Table \d+:.*?(?=\n\n|\Z)', '', text, flags=re.DOTALL)

    """
    Remove in-text citations from the text.
    """
    # Regular expression to find in-text citations, e.g., (Author, Year) or [1]
    text = re.sub(r'\(.*?\)', '', text)  # Removes citations like (Author, Year)
    text = re.sub(r'\[\d+\]', '', text)  # Removes citations like [1], [2], etc.
    return (text)

In [12]:
# Function to preprocess text
def preprocess_text(text):
    """Preprocess the text by tokenizing, removing stopwords, and applying lemmatization and stemming."""
    # Tokenize text
    tokens = word_tokenize(text)

    # Convert to lower case
    tokens = [token.lower() for token in tokens]

    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Stem tokens (optional, often not used with lemmatization)
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    return ' '.join(tokens)


 **Main Function to Process Files in Directory**

In [13]:
# Ensure you have downloaded the necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def process_files_in_directory(input_directory, output_directory):
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"Directory {output_directory} created.")
    else:
        print(f"Directory {output_directory} already exists.")

    # List all PDF files in the input directory
    pdf_files = [file for file in os.listdir(input_directory) if file.endswith('.pdf')]

    for pdf_file in pdf_files:
        pdf_path = os.path.join(input_directory, pdf_file)
        print(f"Reading {pdf_path}...")

        # Extract text from the PDF
        text = extract_text_from_pdf(pdf_path)
        if text is None:
            continue

        # Remove references from the text
        text_without_references = remove_references(text)

        # Clean the text
        cleaned_text = clean_text(text_without_references)

        # Preprocess the text
        preprocessed_text = preprocess_text(cleaned_text)

        # Define the path for the cleaned text file
        clean_file_path = os.path.join(output_directory, os.path.splitext(pdf_file)[0] + '_cleaned.txt')

        # Write the cleaned text to the new file
        with open(clean_file_path, 'w', encoding='utf-8') as file:
            file.write(preprocessed_text)
            print(f"Preprocessed text written to {clean_file_path}")

    print("All files have been processed.")


input_directory = '/content/drive/MyDrive/dataset'  # Replace with your input directory path
output_directory = '/content/drive/MyDrive/Autism'  # Replace with your output directory path

process_files_in_directory(input_directory, output_directory)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Directory /content/drive/MyDrive/Autism already exists.
Reading /content/drive/MyDrive/dataset/zhao2020.pdf...
Preprocessed text written to /content/drive/MyDrive/Autism/zhao2020_cleaned.txt
Reading /content/drive/MyDrive/dataset/Young_Behavior.pdf...
Preprocessed text written to /content/drive/MyDrive/Autism/Young_Behavior_cleaned.txt
Reading /content/drive/MyDrive/dataset/Tariq2018.pdf...
Preprocessed text written to /content/drive/MyDrive/Autism/Tariq2018_cleaned.txt
Reading /content/drive/MyDrive/dataset/Qiu.pdf...
Preprocessed text written to /content/drive/MyDrive/Autism/Qiu_cleaned.txt
Reading /content/drive/MyDrive/dataset/Tariq_2019.pdf...
Preprocessed text written to /content/drive/MyDrive/Autism/Tariq_2019_cleaned.txt
Reading /content/drive/MyDrive/dataset/Patten_Audio.pdf...
Preprocessed text written to /content/drive/MyDrive/Autism/Patten_Audio_cleaned.txt
Reading /content/drive/MyDrive/dataset/Asd_Cry_patterns.pdf...
Preprocessed text written to /content/drive/MyDrive/Aut

**Reading Clean File for Embedding**

In [14]:
documents= SimpleDirectoryReader("/content/drive/MyDrive/Autism").load_data()
print(f"Loaded {len(documents)} documents.")

Loaded 15 documents.


In [13]:
documents

[Document(id_='602aca2e-2b9d-4952-9625-f18713c06082', embedding=None, metadata={'file_path': '/content/drive/MyDrive/Autism/15_Nazneen_cleaned.txt', 'file_name': '15_Nazneen_cleaned.txt', 'file_type': 'text/plain', 'file_size': 0, 'creation_date': '2024-05-31', 'last_modified_date': '2024-05-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='d51f3dde-190b-4c22-89f1-cfd7b38ff62b', embedding=None, metadata={'file_path': '/content/drive/MyDrive/Autism/1_Ramırez-Duque__cleaned.txt', 'file_name': '1_Ramırez-Duque__cleaned.txt', 'file_type': 'text/plain', 'file_size': 30917, 'creation_date': '20

# **Setup default system prompt**

In [15]:
system_prompt= """
You are a Q&A assistant. Your goal is to answer question as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt= SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [16]:
!pip install huggingface-hub

In [17]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

In [18]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16,"load_in_8bit": True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# **Embedding the data**

In [19]:
%pip install llama-index-embeddings-langchain

In [20]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.6 MB/s eta 0:00:00


In [21]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **Chunking the embeddings**

In [22]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-22-6394536e40c5>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [23]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7c7793dc8e20>, num_workers=None), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7c7793dc8e20>, id_func=<function default_id_func at 0x7c7901625b40>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x7c783fb23070>, callback_manager=<llama_index

# **Indexing the embeddings**

In [24]:
index = VectorStoreIndex.from_documents(documents, service_context= service_context,show_progress = True)

Parsing nodes:   0%|          | 0/15 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/107 [00:00<?, ?it/s]

In [25]:
index

# **Creating Query Engine**

In [42]:
query_engine = index.as_query_engine()

In [43]:
response = query_engine.query("What is West Syndrome?")

In [44]:
print(response)

West Syndrome is a rare and severe neurodevelopmental disorder that is characterized by a triad of symptoms: infantile spasms, hypsarrhythmia, and developmental delays. It is usually diagnosed in the first year of life and is associated with a high risk of intellectual disability, autism spectrum disorder, and other neurodevelopmental disorders. The exact cause of West Syndrome is not fully understood, but it is thought to be related to genetic mutations, brain injury, or infections during fetal development or early childhood. Treatment for West Syndrome typically involves a combination of medications and other therapies, such as physical therapy, occupational therapy, and speech therapy, and may also include surgery in some cases. Early diagnosis and treatment are important to improve outcomes for children with West Syndrome.


In [45]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor

In [46]:
retriever= VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor= SimilarityPostprocessor(similarity_cutoff=0.80)

query_engine = RetrieverQueryEngine(retriever=retriever,
                                    node_postprocessors=[postprocessor])

In [47]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source= True)
print(response)

Final Response: West Syndrome is a rare and severe neurodevelopmental
disorder that is characterized by a triad of symptoms: infantile
spasms, hypsarrhythmia, and developmental delays. It is usually
diagnosed in the first year of life and is associated with a high risk
of intellectual disability, autism spectrum disorder, and other
neurodevelopmental disorders. The exact cause of West Syndrome is not
fully understood, but it is thought to be related to genetic
mutations, brain injury, or infections during fetal development or
early childhood. Treatment for West Syndrome typically involves a
combination of medications and other therapies, such as physical
therapy, occupational therapy, and speech therapy, and may also
include surgery in some cases. Early diagnosis and treatment are
important to improve outcomes for children with West Syndrome.
______________________________________________________________________
Source Node 1/2
Node ID: e16a4504-ef60-4ec4-a705-1dd6a887e450
Similarity: 

In [34]:
response = query_engine.query("How can cross country trials help in development of Machine learning based Multimodal solutions 	")